In [3]:
from google.cloud import documentai_v1 as documentai
from google.api_core.client_options import ClientOptions
from google.cloud import storage
import pandas as pd
import time
from pathlib import Path
import json

# === CONFIG ===
project_id = "vercillopersonal"
location = "us"
processor_id = "fe61eee8945a8018"

# === INPUT/OUTPUT PATHS ===
gcs_input_uri = "gs://vercillo_projects/transactions/amex/data/2025/2025-01-03_cleansed.csv"
filename = Path(gcs_input_uri).name               
gcs_output_uri = "gs://vercillo_projects/transactions/amex/exports/"

In [5]:
df = pd.read_csv(gcs_input_uri)
df

,row_id,Vendor,amount,location,posting_date,transaction_date,closing_date,opening_date
0,0,PTZ INSURANCE SERVICES,21.27,OAKVILLE,04-Dec,03-Dec,03-Jan-25,04-Dec-24
1,1,SECURITY NATIONAL INSUR,88.18,MONTREAL,05-Dec,04-Dec,03-Jan-25,04-Dec-24
2,2,UBER EATS,20.14,HTTPS://HELP.UB,05-Dec,05-Dec,03-Jan-25,04-Dec-24
3,3,UBER EATS,1,HTTPS://HELP.UB,05-Dec,05-Dec,03-Jan-25,04-Dec-24
4,4,DC03 A-OK COMMISSARY &,5.65,Vaughan,05-Dec,05-Dec,03-Jan-25,04-Dec-24
...,...,...,...,...,...,...,...,...
62,62,points_redeemed,"26,166",NaN,NaN,NaN,03-Jan-25,04-Dec-24
63,63,PAYMENT RECEIVED,-2481.67,NaN,04-Dec,04-Dec,03-Jan-25,04-Dec-24
64,64,Use Points for Purchases | Reference S00024339...,-261.66,NaN,04-Dec,04-Dec,03-Jan-25,04-Dec-24
65,65,PAYMENT RECEIVED,-1000,NaN,22-Dec,22-Dec,03-Jan-25,04-Dec-24
